In [22]:
import vectorbt as bt 
import pandas as pd
import numpy as np
import pandas_ta as ta

In [23]:
btc_price = bt.YFData.download('BTC-USD', missing_index='drop', start='2024-06-01', end='2024-08-01', interval='1h').get('Close')
print(type(btc_price))
btc_price = btc_price.asfreq('h')

# btc_price is a bt object
# btc_price.data is a dict
# btc_price.data['BTC-USD'] is a pandas dataframe

# btc_price.data['BTC-USD'].Close.info()

<class 'pandas.core.series.Series'>


In [24]:
def custom_indicator(close, short_window=9, long_window=21):

    ema_short = bt.IndicatorFactory.from_pandas_ta('EMA').run(close, length=short_window).ema.to_numpy()
    ema_long = bt.IndicatorFactory.from_pandas_ta('EMA').run(close, length=long_window).ema.to_numpy()
    # adx = bt.IndicatorFactory.from_pandas_ta('ADX').run(close, length = adx_window).adx.to_numpy()
    # rsi = bt.EMA.run(close, window=rsi_window).rsi.to_numpy()
    # ma = bt.MA.run(close, window=ma_window).ma.to_numpy()
    trend = np.where(ema_long > ema_short , -1, 0)
    trend = np.where((ema_short > ema_long), 1, trend)
    return trend

In [25]:
ind = bt.IndicatorFactory(
    class_name='Combination', 
    short_name='comb', 
    input_names=['close'], 
    param_names=['short_window', 'long_window'], 
    output_names=['value']).from_apply_func(
        custom_indicator, 
        short_window=9, long_window=21)

In [26]:
result = ind.run(
    btc_price,
    # short_window=np.arange(1,100,step=2,dtype=int),
    # long_window=np.arange(21,120,step=2,dtype=int),
    short_window=1,
    long_window=87,
    # param_product=True,
)

type(result)

vectorbt.indicators.factory.Combination

In [27]:
# result.value.value_counts()

In [28]:
long = result.value == 1
short = result.value == -1

In [29]:
pf = bt.Portfolio.from_signals(btc_price, entries=long, exits=short)

In [30]:
returns = pf.total_return()

In [31]:
pf.stats()

Start                         2024-05-31 23:00:00+00:00
End                           2024-07-31 22:00:00+00:00
Period                                 61 days 00:00:00
Start Value                                       100.0
End Value                                     98.046695
Total Return [%]                              -1.953305
Benchmark Return [%]                          -3.779236
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              13.789085
Max Drawdown Duration                  56 days 05:00:00
Total Trades                                         35
Total Closed Trades                                  35
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  17.142857
Best Trade [%]                                 9.906104
Worst Trade [%]                               -2

In [32]:
returns = pf.total_return()


In [33]:
pf.plot().show()